In [31]:
import pandas as pd
import cv2
import os
import random
import math
from tqdm import tqdm

In [32]:
video_path = "/home/theo/Documents/Unif/Master/ChimpRec/ChimpRec-Dataset/CCR/videos"
annotations_path = "/home/theo/Documents/Unif/Master/ChimpRec/ChimpRec-Dataset/CCR/metadata/annotations/face_data.csv"
dataset_path = "/home/theo/Documents/Unif/Master/ChimpRec/ChimpRec-Dataset/Face_detection_CCR"

In [ ]:
# proportion of the overall dataset that will be kept
proportion_kept = 0.003 # approximately 1500 images

df = pd.read_csv(annotations_path)

In [34]:
# @input: n: size of the sample
# proportion: (proportion*100)% of the numbers in [0, n-1]
# @outputs
# No numbers in common between the outputs
def segment_dataset(n, proportion):
    # Calculate the number of elements to select based on the proportion
    num_elements = int(n * proportion)
    
    # Generate evenly spaced numbers within the range [0, n-1]
    step = n / num_elements
    selected_numbers = [int(i * step) for i in range(num_elements)]
    
    return selected_numbers

In [ ]:
# @input: path to a video
# @outputs
# Length of the video in terms of frames
def get_len_video(path_to_video):
    video = cv2.VideoCapture(f"{video_path}/{path_to_video}")

    if not video.isOpened():
        print(f"Could not open video: {path_to_video}")
        return 0
    
    length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    video.release()
    return length

In [ ]:
# extracts the annotation data
def extract_metadata(video_type, video_id, frame_number, img_dim):
    sample_id = video_id.replace(".mp4", "")
    sub_df = df.loc[(df["video"] == video_id) & (df["frame"] == frame_number)]
    output_file = f"{dataset_path}/labels/{video_type}/{sample_id}_{frame_number}.txt"

    string_output = ""

    for index, row in sub_df.iterrows():
        string_output += f"\n0 {row['x']} {row['y']} {row['w']} {row['h']}"

    with open(output_file, "w") as file:
        file.write(string_output.strip())
    file.close()
    
    


In [ ]:
# @input:
# type: "train", "val" or "test" --> indicates which part of the dataset it is
# sample_id: name of the video with the extension
# idxs: indexes of the images to keep
# @output:
# nothing, the image are saved in the output folder
def extract_data(type, video_id):

    sample_id = video_id[:-4]

    output_folder = f"{dataset_path}/images/{type}"
    video = cv2.VideoCapture(f"{video_path}/{video_id}")
    len_video = get_len_video(video_id)

    idxs = segment_dataset(len_video, proportion_kept)
    idxs = sorted(idxs)

    for frame_count in idxs:
        df_ = df.loc[df["video"] == video_id]
        df_ = df_.loc[df_["frame"] == frame_count] # .loc[df["label"] != "NOTCHIMP"]

        # Set the video to the specific frame
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
        success, frame = video.read()

        if success and df_.size != 0:
            frame_filename = f"{output_folder}/{sample_id}_{frame_count}.jpg"
            cv2.imwrite(frame_filename, frame)
            
            # extract the annotations
            extract_metadata(type, video_id, frame_count, (frame.shape[1], frame.shape[0]))

    # Release the video capture object
    video.release()

In [ ]:
filenames = os.listdir(video_path)[:2] #only consider the first two videos (can ofc be changed)
n_files = len(filenames)

# training proportion
train_prop = 0.5

numbers = list(range(n_files))
random.shuffle(numbers)

train_video_idxs = numbers[:math.ceil(train_prop*n_files)]
val_video_idxs = numbers[math.ceil(train_prop*n_files):]

def process_dataset(video_idxs, type):
    # progression bar added
    for video_idx in tqdm(video_idxs, desc=f"Processing videos: {type}", colour="green"):
        video_id = filenames[video_idx]
        extract_data(type, video_id)
        

        

In [40]:
process_dataset(train_video_idxs, "train")
process_dataset(val_video_idxs, "val")

Processing videos: val: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it]
